In [2]:
import os
if not (os.path.isfile('data.zip') and os.path.isfile('data.zip')):
    !rm -rf model_discovery
    !rm -rf data
    import gdown
    # download source code
    gdown.download('https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_', output=None, quiet=False)
    # download data
    gdown.download('https://drive.google.com/uc?id=1ozrvNU128WOGWDVhhDKtZENzBMuO-w4i', output=None, quiet=False)

    !unzip -qq data.zip
    !unzip -qq model_discovery.zip

In [2]:
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 71kB 4.4MB/s 


In [3]:
import deepmatcher as dm
import itertools
import os
import pandas as pd
import model_discovery
from tqdm.notebook import tqdm

In [5]:
# download fasttext word vector
gdown.download('https://drive.google.com/uc?id=12tdZ76mb8tT98pz3lAy2gWIPZvmKDTSo', output=None, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=12tdZ76mb8tT98pz3lAy2gWIPZvmKDTSo
To: /content/wiki.en.bin
8.49GB [01:49, 77.7MB/s]


'wiki.en.bin'

In [10]:
WIKI_EN_PATH = '/content'
DATA_PATH = 'data/deepmatcher/'
datasets = ['abt_buy_exp', 'dblp_acm_exp_data', 'dblp_scholar_exp_data',
       'dirty_itunes_amazon_exp_data', 'walmart_amazon_exp_data']

In [12]:
def model_serving(src, tar):
  src_file_train = ''.join([src, '-train.csv'])
  src_file_valid = ''.join([src, '-valid.csv'])
  tar_file_valid = ''.join([tar, '-train.csv'])

  train, validation, test = dm.data.process(
                  path=DATA_PATH,
                  train=src_file_train,
                  validation=src_file_valid,
                  test=tar_file_valid,
                  embeddings_cache_path=WIKI_EN_PATH)
  
  model = dm.MatchingModel(attr_summarizer='hybrid')

  # number of epochs can be changed to a higher value to obtain better accuracy
  train_result = model.run_train(
            train,
            validation,
            epochs=10,
            batch_size=16,
            best_save_path='hybrid_model.pth',
            pos_neg_ratio=3)
  serving_result = model.run_eval(test)

  return train_result.item(), serving_result.item()

In [13]:
comb = list(itertools.permutations(datasets, 2))
src_list = []
tar_list = []
src_acc_list = []
ser_acc_list = []
for src, tar in tqdm(comb, leave=False):
  train_result, serving_result = model_serving(src, tar)

  src_list.append(src)
  tar_list.append(tar)
  src_acc_list.append(train_result)
  ser_acc_list.append(serving_result)


Reading and processing data from "data/deepmatcher/abt_buy_exp-train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "data/deepmatcher/abt_buy_exp-valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "data/deepmatcher/dblp_acm_exp_data-train.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:770: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_inpu

* Number of trainable parameters: 5056204
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2611: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Finished Epoch 1 || Run Time:   33.0 | Load Time:   11.6 || F1:  31.99 | Prec:  37.75 | Rec:  27.76 || Ex/s: 128.85

===>  EVAL Epoch 1


0% [████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


...

Finished Epoch 10 || Run Time:   30.5 | Load Time:    5.7 || F1:  93.46 | Prec:  90.42 | Rec:  96.70 || Ex/s: 170.05

===>  EVAL Epoch 10


0% [█████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 10 || Run Time:    5.1 | Load Time:    1.9 || F1:  49.49 | Prec:  48.28 | Rec:  50.78 || Ex/s: 292.90

---------------------

Loading best model...
Training done.
===>  EVAL Epoch 8


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 8 || Run Time:    0.5 | Load Time:    0.6 || F1:  50.68 | Prec:  54.41 | Rec:  47.44 || Ex/s: 281.52



In [14]:
result_df = pd.DataFrame({'Source 1': src_list, 'Source 2': tar_list, 
              'Source Accuracy': src_acc_list, 'Serving Accuracy': ser_acc_list})

result_df.to_csv(base_dir+'/dm_serving-train.csv', index=False)

In [15]:
result_df

,Source 1,Source 2,Source Accuracy,Serving Accuracy
0,abt_buy_exp,dblp_acm_exp_data,67.000000,35.683094
1,abt_buy_exp,dblp_scholar_exp_data,66.836731,38.968227
2,abt_buy_exp,dirty_itunes_amazon_exp_data,67.153282,42.140469
3,abt_buy_exp,walmart_amazon_exp_data,68.705879,23.683445
4,dblp_acm_exp_data,abt_buy_exp,95.575218,20.894022
5,dblp_acm_exp_data,dblp_scholar_exp_data,96.444443,83.800003
6,dblp_acm_exp_data,dirty_itunes_amazon_exp_data,96.132591,61.437908
7,dblp_acm_exp_data,walmart_amazon_exp_data,95.535713,21.081081
8,dblp_scholar_exp_data,abt_buy_exp,93.012489,20.395155
9,dblp_scholar_exp_data,dblp_acm_exp_data,92.370201,95.297112
